In [119]:

USE ERP
GO
CREATE PROCEDURE usp_DropDatabase
    @DatabaseName NVARCHAR(128)
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @KillCommand NVARCHAR(MAX);
    DECLARE @Sql NVARCHAR(MAX);

    -- Initialize the command to kill active connections
    SET @KillCommand = '';

    -- Build the kill command for each active connection, excluding the current session
    SELECT @KillCommand = @KillCommand + 'KILL ' + CONVERT(NVARCHAR(10), session_id) + ';'
    FROM sys.dm_exec_sessions
    WHERE database_id = DB_ID(@DatabaseName) AND session_id <> @@SPID;

    -- Execute the kill commands
    EXEC sp_executesql @KillCommand;

    -- Drop the database
    SET @Sql = N'DROP DATABASE IF EXISTS [' + @DatabaseName + ']';
    EXEC sp_executesql @Sql;
END

GO

/*
Anlegen der grundsätzlichen Datenbankobjekte (Datenbank, Schema)
*/

-- Ausführen der Helper Stored Procedure um die Datenbank zu löschen
EXEC usp_DropDatabase 'dwh'

CREATE DATABASE dwh COLLATE SQL_Latin1_General_CP1_CI_AS; -- Gleiche Collation wie die ERP Datenbank


Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure usp_DropDatabase, Line 1
There is already an object named 'usp_DropDatabase' in the database.

Commands completed successfully.

Total execution time: 00:00:00.647

In [120]:
USE DWH;
GO

CREATE PROCEDURE Transfer_TopSeller
(
    @SourceDB VARCHAR(50) = 'ERP',
    @SourceTable1 VARCHAR(50) = 'Sales.SalesOrderHeader',
    @DestinationTable1 VARCHAR(50) = 'Staging_TopSeller.SalesOrderHeader',
    @SourceTable2 VARCHAR(50) = 'Sales.SalesOrderDetail',
    @DestinationTable2 VARCHAR(50) = 'Staging_TopSeller.SalesOrderDetail',
    @SourceTable3 VARCHAR(50) = 'Sales.SalesTerritory',
    @DestinationTable3 VARCHAR(50) = 'Staging_TopSeller.SalesTerritory',
    @SourceTable4 VARCHAR(50) = 'Sales.SalesPerson',
    @DestinationTable4 VARCHAR(50) = 'Staging_TopSeller.SalesPerson',
    @DestinationDB VARCHAR(50) = 'DWH'
)
AS
BEGIN
    DECLARE @SQL NVARCHAR(MAX);
    DECLARE @FullyQualifiedDestinationTable NVARCHAR(128);

    BEGIN TRY
        -- Erste Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable1;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable1;
        EXEC sp_executesql @SQL;

        -- Zweite Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable2;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable2;
        EXEC sp_executesql @SQL;

        -- Dritte Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable3;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable3;
        EXEC sp_executesql @SQL;

        -- Vierte Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable4;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable4;
        EXEC sp_executesql @SQL;

        -- Fünfte Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable5;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END


    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.131

In [ ]:
USE DWH;
GO

CREATE PROCEDURE CreateSalespersonIDTable
AS
BEGIN
    SET NOCOUNT ON;

    BEGIN TRY
        -- Überprüfen, ob die Tabelle existiert, und Löschen, falls sie vorhanden ist
        IF OBJECT_ID('DWH.DWH_TopSeller.Dim_SalesPersonID', 'U') IS NOT NULL
        BEGIN
            DROP TABLE DWH.DWH_TopSeller.Dim_SalesPersonID; 
        END

        -- Erstellen der Tabelle Dim_SalesPersonID mit den Spalten SalesPersonID und SalesPersonName
        CREATE TABLE [DWH].[DWH_TopSeller].Dim_SalesPersonID (
            SalesPersonID INT PRIMARY KEY NOT NULL,      
            SalesPersonName VARCHAR(100)
        );

        -- Einfügen der Werte in die Tabelle
        INSERT INTO [DWH].[DWH_TopSeller].Dim_SalesPersonID (SalesPersonID, SalesPersonName)
        VALUES 
            (279, 'Stephen Oswald'),
            (277, 'John Doe'),
            (283, 'Alice Smith'),
            (276, 'Bob Johnson'),
            (282, 'Charlie Brown'),
            (275, 'Diana White'),
            (290, 'Eve Black'),
            (281, 'Frank Green'),
            (288, 'Grace Lee'),
            (289, 'Harry Martin'),
            (280, 'Irene Davis'),
            (286, 'Jack Wilson');
    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;
GO


In [121]:
USE DWH;
GO
CREATE PROCEDURE Transfer_SalesGrowth
(
    @SourceDB VARCHAR(50) = 'ERP',
    @SourceTable1 VARCHAR(50) = 'Sales.SalesOrderHeader',
    @DestinationTable1 VARCHAR(50) = 'Staging_SalesGrowth.SalesOrderHeader',
    @SourceTable2 VARCHAR(50) = 'Sales.SalesOrderDetail',
    @DestinationTable2 VARCHAR(50) = 'Staging_SalesGrowth.SalesOrderDetail',
    @SourceTable3 VARCHAR(50) = 'Production.Product',
    @DestinationTable3 VARCHAR(50) = 'Staging_SalesGrowth.Product',
    @DestinationDB VARCHAR(50) = 'DWH'
)
AS
BEGIN
    DECLARE @SQL NVARCHAR(MAX);
    DECLARE @FullyQualifiedDestinationTable NVARCHAR(128);

    BEGIN TRY
        -- Erste Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable1;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable1;
        EXEC sp_executesql @SQL;

        -- Zweite Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable2;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable2;
        EXEC sp_executesql @SQL;

        -- Dritte Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable3;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable3;
        EXEC sp_executesql @SQL;
    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.122

In [122]:
USE DWH;
GO

CREATE PROCEDURE ETL_TopSeller_ProductID
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @TableToCreate NVARCHAR(128) = 'DWH.DWH_SalesGrowth.Dim_ProductID';
    DECLARE @SQL NVARCHAR(MAX);

    BEGIN TRY

    IF OBJECT_ID(@TableToCreate, 'U') IS NOT NULL
    BEGIN
        SET @SQL = 'DROP TABLE ' + @TableToCreate;
        EXEC sp_executesql @SQL;
    END

    SET @SQL = 'CREATE TABLE ' + @TableToCreate + ' (
        ProductID INT PRIMARY KEY,
        Produktname VARCHAR(MAX)
    )';
    EXEC sp_executesql @SQL;

-- Define the SELECT DISTINCT AND THE COLUMNS we want to select
    SET @SQL = 'INSERT INTO ' + @TableToCreate + ' (ProductID, Produktname)
    SELECT DISTINCT pr.ProductID, 
pr.Name as Produktname
FROM  
    Staging_SalesGrowth.SalesOrderHeader sh  
LEFT OUTER JOIN  
    Staging_SalesGrowth.SalesOrderDetail sd ON sh.SalesOrderID = sd.SalesOrderID 
LEFT OUTER JOIN  
    Staging_SalesGrowth.Product pr ON sd.ProductID = pr.ProductID';
    EXEC sp_executesql @SQL;

    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.173

In [123]:
USE DWH;
GO

CREATE PROCEDURE "ETL_TopSeller_TerritoryID"
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @TableToCreate NVARCHAR(128) = 'DWH.DWH_TopSeller.Dim_TerritoryID';
    DECLARE @SQL NVARCHAR(MAX);

    BEGIN TRY

    IF OBJECT_ID(@TableToCreate, 'U') IS NOT NULL
    BEGIN
        SET @SQL = 'DROP TABLE ' + @TableToCreate;
        EXEC sp_executesql @SQL;
    END

    SET @SQL = 'CREATE TABLE ' + @TableToCreate + ' (
        territoryID INT PRIMARY KEY,
        Territory VARCHAR(MAX)
    )';
    EXEC sp_executesql @SQL;

    SET @SQL = 'INSERT INTO ' + @TableToCreate + ' (territoryID, Territory)
    SELECT DISTINCT sap.territoryID, 
st.Name as Territory
from Staging_TopSeller.SalesPerson sap 
left outer join Staging_TopSeller.SalesTerritory st on st.territoryID = sap.TerritoryID
WHERE sap.territoryID IS NOT NULL';
    EXEC sp_executesql @SQL;
    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.193

In [124]:
USE DWH;
GO

CREATE PROCEDURE "ETL_TopSeller_SalesPersonID"
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @TableToCreate NVARCHAR(128) = 'DWH.DWH_TopSeller.Dim_SalesPersonID';
    DECLARE @SQL NVARCHAR(MAX);

    BEGIN TRY

    IF OBJECT_ID(@TableToCreate, 'U') IS NOT NULL
    BEGIN
        SET @SQL = 'DROP TABLE ' + @TableToCreate;
        EXEC sp_executesql @SQL;
    END

    SET @SQL = 'CREATE TABLE ' + @TableToCreate + ' (
        SalesPersonID INT PRIMARY KEY,
        Name VARCHAR(MAX)
    )';
    EXEC sp_executesql @SQL;

    SET @SQL = 'INSERT INTO ' + @TableToCreate + ' (SalesPersonID, Name)
    SELECT DISTINCT sap.SalesPersonID, 
sap.Name as Name
from DWH.Staging_TopSeller.Store sap ';    
    EXEC sp_executesql @SQL;
    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.104

In [125]:
CREATE SCHEMA Staging_TopSeller

Commands completed successfully.

Total execution time: 00:00:00.055

In [126]:
CREATE SCHEMA Staging_SalesGrowth

Commands completed successfully.

Total execution time: 00:00:00.062

In [127]:
CREATE SCHEMA DWH_SalesGrowth

Commands completed successfully.

Total execution time: 00:00:00.056

In [128]:
CREATE SCHEMA DWH_TopSeller

Commands completed successfully.

Total execution time: 00:00:00.080

In [129]:
EXEC Transfer_TopSeller;
EXEC Transfer_SalesGrowth
EXEC CreateSalespersonIDTable
EXEC ETL_TopSeller_ProductID
EXEC ETL_TopSeller_TerritoryID
EXEC ETL_TopSeller_SalesPersonID

(31465 rows affected)

(121317 rows affected)

(10 rows affected)

(17 rows affected)

Ein Fehler ist aufgetreten: Invalid object name 'DWH.Staging_TopSeller.Store'.

: Msg 208, Level 16, State 1, Line 2
Invalid object name 'DWH.Staging_TopSeller.Store'.

Total execution time: 00:00:01.446

In [118]:
EXEC ETL_TopSeller_SalesPersonID


Ein Fehler ist aufgetreten: Invalid object name 'Staging_TopSeller.Store'.

: Msg 208, Level 16, State 1, Line 1
Invalid object name 'Staging_TopSeller.Store'.

Total execution time: 00:00:00.112